In [1]:
import warnings
warnings.filterwarnings("ignore")

from DataUtils import *
from hfUtils import *

from tqdm.auto import tqdm

import numpy as np
import torch

# First Load The Data to RAM
Below is the function to call to get the data loaded and formatted into prompts.

In [2]:
train_prompts, test_prompts, subjects = load_mmlu_to_ram('./MMLU')

print some examples

In [3]:
n_examples = 2
n_smaples = 1

for i in range(n_examples):
    rnd_subject = np.random.choice(len(subjects))
    
    print('Subject:', subjects[rnd_subject])
    print("--------------------")
    print("Training Prompt:")
    print(train_prompts[rnd_subject])
    print('--------------------')
    
    for j in range(n_smaples):
        rnd_question = np.random.choice(len(test_prompts[rnd_subject]))
        
        print('Test Question:', test_prompts[rnd_subject][rnd_question][0])
        print('Expected Answer:', test_prompts[rnd_subject][rnd_question][1])

Subject: professional psychology
--------------------
Training Prompt:
The following are multiple choice questions (with answers) about professional psychology.

One of your therapy clients asks your advice about a good weight- reduction program. You have investigated the programs in the community and are enrolled in the one you consider the best. This program offers a $50 bonus to its patrons for each new person they bring into the program. Under these circumstances, your most appropriate response would be to
A. tell your client the pros and cons of each program you know about except for the one in which you are enrolled
B. recommend to your client the program in which you are enrolled and explain the $50 bonus you will receive
C. recommend to your client the program in which you are enrolled and offer to have the $50 bonus credited to your client's account in the program
D. tell your client the pros and cons of each program you know about, but do not claim the $50 bonus if your clien

# Testing A Model
For now let's test google/gemma-2b-it

In [4]:
tokenizer, model = load_model_and_tokenizer("google/gemma-2b-it")

2024-11-10 17:42:49.683750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 17:42:49.689879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731278569.696750 3314381 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731278569.698786 3314381 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 17:42:49.706251: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
predictions[0]

['C',
 'A',
 'B',
 'A',
 'B',
 'B',
 'A',
 'A',
 'C',
 'C',
 'A',
 'C',
 'B',
 'A',
 'B',
 'A',
 'C',
 'B',
 'C',
 'A',
 'A',
 'B',
 'A',
 'C',
 'A',
 'C',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'C',
 'C',
 'A',
 'A',
 'B',
 'A',
 'B',
 'C',
 'A',
 'A',
 'A',
 'B',
 'B',
 'A',
 'B',
 'A',
 'B',
 'A',
 'A',
 'B',
 'A',
 'A',
 'A',
 'C',
 'A',
 'A',
 'A',
 'A',
 'C',
 'C',
 'A',
 'A',
 'A',
 'A',
 'A',
 'C',
 'C',
 'C',
 'B',
 'A',
 'A',
 'A',
 'B',
 'A',
 'A',
 'C',
 'A',
 'A',
 'A',
 'A',
 'C',
 'B',
 'A',
 'C',
 'B',
 'A',
 'B',
 'C',
 'B',
 'B',
 'A',
 'B',
 'B',
 'A',
 'A']

In [23]:
# run all test prompts through the model and get the predictions
with torch.no_grad():
    predictions = []
    accuracies = []
    for i,subject in enumerate(subjects):
        train_prompt = train_prompts[i]
        predictions.append([])
        accuracies.append(0)
        for j in tqdm(range(len(test_prompts[i]))):
            input_prompt = train_prompt + test_prompts[i][j][0]
            expected_output = test_prompts[i][j][1]
            
            input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to(model.device)
            logits = model(input_ids=input_ids).logits[0, -1]
            
            probs = (
                torch.nn.functional.softmax(
                    torch.tensor(
                        [
                            logits[tokenizer("A").input_ids[-1]],
                            logits[tokenizer("B").input_ids[-1]],
                            logits[tokenizer("C").input_ids[-1]],
                            logits[tokenizer("D").input_ids[-1]],
                        ]
                    ).float(),
                    dim=0,
                )
                .detach()
                .cpu()
                .numpy()
            )
            pred = {0: "A", 1: "B", 2: "C", 3: "D"}[np.argmax(probs)]
        
            # output = model.generate(input_ids, max_length=input_ids.shape[1]+1, num_return_sequences=1, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
            # prediction = tokenizer.decode(output[0], skip_special_tokens=True)[len(input_prompt):]
            
            predictions[-1].append(pred)
            
            if pred == expected_output:
                accuracies[-1] += 1
                
        accuracies[-1] /= len(test_prompts[i])
        print(f"Subject {subject} accuracy: {accuracies[-1]}")

  0%|          | 0/100 [00:00<?, ?it/s]

Subject abstract algebra accuracy: 0.0


  0%|          | 0/135 [00:00<?, ?it/s]

Subject anatomy accuracy: 0.0


  0%|          | 0/152 [00:00<?, ?it/s]

Subject astronomy accuracy: 0.0


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# print out sample predictions

MIT stands for Massachusetts Institute of Technology. It is a private research university in Cambridge, Massachusetts, United States. MIT is one of the world's leading research universities, with a wide range of programs in science, technology, engineering, and medicine.


In [ ]:
n_examples = 2
n_smaples = 1

for i in range(n_examples):
    rnd_subject = np.random.choice(len(subjects))
    
    print('Subject:', subjects[rnd_subject])
    print("--------------------")
    print("Training Prompt:")
    print(train_prompts[rnd_subject])
    print('--------------------')
    
    for j in range(n_smaples):
        rnd_question = np.random.choice(len(test_prompts[rnd_subject]))
        
        print('Test Question:', test_prompts[rnd_subject][rnd_question][0])
        print('Expected Answer:', test_prompts[rnd_subject][rnd_question][1])
        print('Predicted Answer:', predictions[rnd_subject][rnd_question])